In [1]:
!pip install split_folders

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import splitfolders

In [8]:
input_folder="stage2_mask"
output="Knee_Mask_Dataset/processed_data"
splitfolders.ratio(input_folder,output, seed=42, ratio=(.7,.15,.15))

Copying files: 5686 files [00:33, 171.19 files/s]


In [9]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

In [7]:
img_height, img_width = (224, 224)
batch_size = 32
train_data_dir = r"Knee_Mask_Dataset\processed_data\train"
valid_data_dir = r"Knee_Mask_Dataset\processed_data\val"
test_data_dir = r"Knee_Mask_Dataset\processed_data\test"

In [44]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  validation_split=0.4)

In [45]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

Found 2389 images belonging to 5 classes.


In [46]:
valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')# set as validation data

Found 338 images belonging to 5 classes.


In [47]:
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    subset='validation')# set as validation data

Found 341 images belonging to 5 classes.


In [48]:
x,y = test_generator.next()
x.shape

(1, 224, 224, 3)

In [49]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
  layer.trainable = False

In [21]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics = ['accuracy'])
model.fit(train_generator,epochs = 100)

Epoch 1/100
75/75 [==============================] - 174s 2s/step - loss: 1.3158 - accuracy: 0.4094
Epoch 2/100
75/75 [==============================] - 153s 2s/step - loss: 1.2773 - accuracy: 0.4395
Epoch 3/100
75/75 [==============================] - 142s 2s/step - loss: 1.2700 - accuracy: 0.4508
Epoch 4/100
75/75 [==============================] - 148s 2s/step - loss: 1.2529 - accuracy: 0.4496
Epoch 5/100
75/75 [==============================] - 141s 2s/step - loss: 1.2729 - accuracy: 0.4496
Epoch 6/100
75/75 [==============================] - 144s 2s/step - loss: 1.2502 - accuracy: 0.4546
Epoch 7/100
75/75 [==============================] - 143s 2s/step - loss: 1.2358 - accuracy: 0.4567
Epoch 8/100
75/75 [==============================] - 143s 2s/step - loss: 1.2336 - accuracy: 0.4617
Epoch 9/100
75/75 [==============================] - 142s 2s/step - loss: 1.2359 - accuracy: 0.4521
Epoch 10/100
75/75 [==============================] - 143s 2s/step - loss: 1.2215 - accuracy: 0.4621

In [94]:
import pickle
with open('model_KOA_Resnet','wb') as f:
    pickle.dump(model,f)

In [2]:
import pickle
with open('model_KOA_Resnet','rb') as f:
    model = pickle.load(f)

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('\nTest accuracy',test_acc)

In [16]:
import cv2

In [17]:
image = cv2.imread('C:/Users/sahil/Unet Implementation/Knee Osteoarthritis Cartilage Segementation Using Unet/Knee_Mask_Dataset/processed_data/train/Grade4/9910391R.png')

In [18]:
image_resized = cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)

(1, 224, 224, 3)


In [19]:
pred=model.predict(image)
print(pred)

1/1 [==============================] - 1s 935ms/step
[[0.25898114 0.26289123 0.28673235 0.06548575 0.12590958]]


In [20]:
class_names = ["Grade 0", "Grade 1", "Grade 2", "Grade 3", "Grade 4"]

In [21]:
output_class = class_names[np.argmax(pred)]
print("The predicted class is ",output_class)

The predicted class is  Grade 2
